In [1]:
import numpy as np

In [2]:
num_pulleys = 4

$$ l_i = L_i + r_i\Delta\theta_i $$

In [3]:
def get_Li(B, D):
    """
    The inital length of strings based on B and D
    """
    return 0.5*np.sqrt(B**2 + D**2)

### TODO
 - [ ] We need to account for 360 degree rotation here
    - Are you talking about the decrease in radius each time the pulley rotates completely?
 - [x] We need to have a reverse calculation done, i.e. given a $x_i,y_i,z_i$ & $x_f,y_f,z_f$ we need to find out what the inital and final thetas. But more on that later.

In [4]:
def get_final_string_length(Li, r, delta_thetas=np.zeros((1,num_pulleys))):
    """
    The final length of strings which need to be achieved for 
    reaching a specific location (based on thetas).
    """
    return Li + r*delta_thetas

def get_theta_from_string_length(li, Li, r):
    """
    Given some string lengths li and initial lengths Li, return
    the rotation of the pulleys
    """
    return (li - Li)/r

In [5]:
def get_coordinate(thetas, B, D, H, r):
    """
    thetas: numpy array of rotations, shape: (num_arrays, )
    B, D, H, r: scalar constants
    """
    Li = get_Li(B, D)
    l = get_final_string_length(Li, r, thetas)
    x = (l[1]**2 - l[2]**2 + B**2)/(2*B)
    y = (l[0]**2 - l[1]**2 + D**2)/(2*D)
    Hz2 = l[0]**2 - x**2 - y**2 # (H-z)^2, eqn 1
    # Is it taut?
    # print(l[0]**2 - l[3]**2, l[1]**2 - l[2]**2)
    z = H - np.sqrt(Hz2)
    return np.array([x, y, z])

In [6]:
def get_thetas(coords, B, D, H, r):
    """
    coords: (x, y, z) coordinates
    B, D, H, r: scalar constants
    """
    l = np.zeros((4,))
    x, y, z = coords
    Hz2 = (H - z)**2 # common term
    l[0] = np.sqrt(x**2 + y**2 + Hz2)
    l[1] = np.sqrt(x**2 + (D - y)**2 + Hz2)
    l[2] = np.sqrt((B - x)**2 + (D - y)**2 + Hz2)
    l[3] = np.sqrt((B - x)**2 + y**2 + Hz2)
    return get_theta_from_string_length(l, get_Li(B, D), r)

In [7]:
# Box dimensions
B, D, H = 100, 200, 300
# Pulley radius
r = 0.1
# Pulley locations
Pulleys = [(0, 0, H), (0, D, H), (B, D, H), (B, 0, H)]

Taking some random values of $\Theta$, we can get the corresponding coordinates from `get_coordinates()`.
If $\Theta$ is such that the ropes are taught, we will get back the original $\Theta$ from `get_thetas()`.
If they aren't, the values will be different. Both are demonstrated in the following example. In the first case, $\theta_4$ is clearly different from the value we set, but in the second case, all angles are exactly the same.

In [8]:
def display(rotations, coords, theta):
    print(f'For rotation = {rotations/(2*np.pi)}')
    x, y, z = coords
    print(f'\tx = {x}, y = {y}, z = {z}')
    print('\tΘ = ', theta)
    print()
    
# Pulleys rotated 10 revolutions, 20, 30, and 40, respectively
rotations = np.array([10, 20, 30, 40])*2*np.pi
coords = get_coordinate(rotations, B, D, H, r)
theta = get_thetas(coords, B, D, H, r)/(2*np.pi)
display(rotations, coords, theta)

# Pulleys rotated 10 revolutions, 20, 20, and 10, respectively
rotations = np.array([10, 20, 20, 10])*2*np.pi
coords = get_coordinate(rotations, B, D, H, r)
theta = get_thetas(coords, B, D, H, r)/(2*np.pi)
display(rotations, coords, theta)

For rotation = [10. 20. 30. 40.]
	x = 41.98822482885034, y = 96.19150450244695, z = 245.88322994588324
	Θ =  [10.        20.        30.        20.5045589]

For rotation = [10. 20. 20. 10.]
	x = 50.0, y = 96.19150450244695, z = 253.18508971098808
	Θ =  [10. 20. 20. 10.]



## A plot of the ropes
### Note
The following blocks need Jupyter Lab with [the `jupyter-matplotlib` extension](https://github.com/matplotlib/jupyter-matplotlib#installation) installed. This allows interactive plots, so we can pan and zoom on the 3D plots and get more interesting viewpoints.

In [9]:
%matplotlib widget 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

x, y, z = coords

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for p in Pulleys:
    ax.plot((p[0], x), (p[1], y), zs=[p[2], z])

ax.set_xlim3d(0, B)
ax.set_ylim3d(0, D)
ax.set_zlim3d(200, H)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# manually tweaked to get a nicer POV
ax.view_init(elev=4.5, azim=-16)
ax.dist = 7.5

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## An animation of movement of the robot

As stated in the Equations notebook, any motion of the robot is the result a sequence of steps of the pulley stepper motors. We can generate a sequence of steps for each pulley where each step is one of:

- $1$, indicating the pulley rotated forward one step
- $0$, indicating the pulley didn't rotate
- $-1$, indicating the pulley rotated backward one step

Then, if the pulley rotates by $d\theta_{step}$ for each step, we can get sequence of $d\Theta$s, and taking some random initial $\Theta$, a sequence of $\Theta$.

Let us plot the $(x, y, z)$ for each $\Theta$.

As example, the following function rotates two pulleys every step, and two pulleys every other step. So the first two pulleys get rotated 2x more than the other two pulleys. The effect is that the robot traces an arc.

This is how the final frame of the animation should look like:

![](arc.png)

In [10]:
def get_random_steps(N_steps):
    """
    Example stepper. Each step can be one of three values:
    rotate forward (1), don't move (0) or rotate back (-1)
    """
    for i in range(N_steps):
        yield(np.array([1, 1, i%2, i%2]))


dtheta_step = 10*(np.pi/180) # say, 10°
r = 0.1 # radius of pulley

np.random.seed(17648) # fixed seed to generate same steps each time

N_steps = 100
Theta = np.random.randint(3*np.pi, 5*np.pi, (4,))
Point = get_coordinate(Theta, B, D, H, r)

Thetas = [Theta]
Points = [Point]
steps = []
for step in get_random_steps(N_steps):
    steps.append(step)
    # since steps are too small to show in the figure,
    # let's scale the steps by a factor of 5
    dtheta = (dtheta_step*step)*5
    Theta = Thetas[-1] + dtheta
    Point = get_coordinate(Theta, B, D, H, r)
    Points.append(Point)
    Thetas.append(Theta)

In [11]:
import mpl_toolkits.mplot3d.axes3d as p3

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Initial settings for the plot
P0 = Points[0]
lines = [ax.plot((P[0], P0[0]), (P[1], P0[1]), zs=[P[2], P0[2]])[0] for P in Pulleys]
path = ax.plot((P0[0], P0[0]), (P0[1], P0[1]), zs=[P0[2], P0[2]])[0]

# Creating the animation function and object
def update_lines(num, Points, lines, path):
    Point = Points[num]
    for i in range(num_pulleys):
        # Redraw each rope
        line = lines[i]
        Pulley = Pulleys[i]
        line.set_data_3d((Pulley[0], Point[0]),
                         (Pulley[1], Point[1]),
                         (Pulley[2], Point[2]))
    # Trace the path of the robot
    x, y, z = path.get_data_3d()
    if num == 0:
        path.set_data_3d([Point[0]], [Point[1]], [Point[2]])
    else:
        path.set_data_3d(np.append(x, Point[0]),
                         np.append(y, Point[1]),
                         np.append(z, Point[2]))
    return lines
line_ani = animation.FuncAnimation(fig, update_lines, N_steps, fargs=(Points, lines, path),
                                   repeat=False, blit=False)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim3d(0, B)
ax.set_ylim3d(0, D)
ax.set_zlim3d(250, H)
# manually tweaked to get a nicer POV
ax.view_init(elev=18, azim=-70)
ax.dist = 7.5

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …